# The Data Preprocessing

In [100]:
import pandas as pd 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  #reading the wikipedia page for getting postal details
df_list = pd.read_html(url)

In [101]:
len(df_list)

3

In [102]:
df_canada_post=df_list[0] #contains the table 
print(len(df_canada_post))#length of the dataframe
df_canada_post.head()

180


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [103]:
df_canada=df_canada_post[df_canada_post["Borough"]!="Not assigned"].reset_index(drop=True) # Removing the Not assigned rows from DataFrame
df_canada.head()# and resting the index

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [104]:
postal_df=pd.read_csv("http://cocl.us/Geospatial_data")#reading the given CSV file for getting the lat and log details
postal_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [109]:
column_names=[col for col in df_canada.columns]+["Latitude","Longitude"] #intializing the new headers for obtain the required dataframe
column_names

['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [108]:
df_canada_Postal = pd.DataFrame(columns=column_names)#Create an empty dataframe with mentioned headers
df_canada_Postal

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [91]:
#funtion defined for obtaining the lattitude and Longitude
def get_lat_log(postal_code):
    x=postal_df.index[postal_df["Postal Code"]== postal_code].item()
    return postal_df.loc[x,"Latitude"],postal_df.loc[x,"Longitude"]    

In [92]:
#Created reqi
for Postal_Code,Borough,Neighbourhood in zip(df_canada["Postal Code"],df_canada["Borough"],df_canada["Neighbourhood"]):
    Latitude=get_lat_log(Postal_Code)[0]
    Longitude=get_lat_log(Postal_Code)[1]
    df_canada_Postal=df_canada_Postal.append({"Postal Code":Postal_Code,
                             "Borough":Borough,
                             "Neighbourhood":Neighbourhood,
                             "Latitude":Latitude,
                             "Longitude":Longitude},ignore_index=True)
df_canada_Postal

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
